<center>

![IGAC](data/images/image_igac.jpg)

</center>

---
## ***Observatorio inmobiliario catastral***

### ***Curso introducción al lenguaje de programación `python`.***

### ***Lectura 4.***
---


Instructor : *_[Luis Andres Campos Maldonado](https://co.linkedin.com/in/lacamposm)_*

---

##  ***Automatizando procesos para la base catastral `R1`.***

---

#### ***Problema:  Desarrollar un análisis descriptivo básico de la base catastral `R1`.***

Debemos responder las preguntan que sean necesarias para determinar si este proceso depende o no del munucipio, es caso que la respuesta sea `NO`, tenemos la opción de automatizarlo.

Solo algunas preguntas:

1. ¿Todos los municipio tiene la misma forma de la base catastral `R1`? 
2. ¿Siempre recibimos la misma forma de la información? 
3. ¿Está en el mismo formato?


Pasos que debemos seguir para automatizar el proceso:

1. Abrir el archivo.
2. Transformalo en un `pd.Dataframe`
3. Manipular el dataframe y generar un nuevo `pd.DataFrame` "mejorado".
4. Generar las estadísticas descriptivas de interés a nivel tabular.
5. Generar estadísticas descriptivas a nivel gráfico básico.

#### ***Vamos a desarrollar una class en python con 3 métodos:***

1. Escribir el `pd.DataFrame` "mejorado" en un archivo Excel.
2. Descriptivos tabulares generales.
3. Descriptivos gráficos básicos.

In [1]:
import os
import bz2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly.subplots import make_subplots


class ManipulationPyR1():
    """
    """
    temp = pd.read_excel('data/DESTINACION_ECONOMICA_CODIGOS.xlsx')
    MAPPER_DESTI_ECONO = dict(zip(temp.CODIGO.astype('string'), temp['NOMBRE DE DESTINACIÓN ECONÓMICA']))
    MAPPER_CONDITION = {'0':'NoRegPH', '2':'PredioInformal', '3':'UsoPubNoVias', '4':'Vias', '5':'MejoraPredioAjeno',
                        '6':'MejAjenoPH', '7':'ParqCemente', '8':'PH_Condominio', '9':'PH'}
    ###
    def __init__(self, path_file:str):
        self.path_file = os.path.abspath(path_file)
    ###
    def _clean_pandas_dataframe(self):
        """
        """
        #with bz2.BZ2File(self.path_file) as f:
        #    dataframe = pd.read_csv(f, sep='\t')
        ## Caso csv
        dataframe = pd.read_csv(self.path_file)
        def zone_classification(zone):
            """
            Esta función reasigna la zona en tres grandes grupos: rural, urbano y corregimientos
            Input: str (00, 01, 02, ..., 09)  <--- SE PODRÍA PENSAR EN UN mapper como el caso anterior.
            Output: str con los códigos en palabras.
            """
            if zone == '00':
                return 'rural'
            elif zone == '01':
                return 'urbano'
            else:
                return 'corregimiento'
        ####
        dataframe['NPN'] = dataframe['DEPARTAMENTO'].astype('str')+dataframe['MUNICIPIO'].astype('str') + \
                           dataframe['NUMERO_DEL_PREDIO']  
        column = dataframe.pop('NPN')            ## Se obtiene toda la pd.Series asociada a la columna "NPN"
        dataframe.insert(0, 'NPN', column)       ## Insertamos la columna "NPN" como la primera.
        dataframe.drop(columns=['DEPARTAMENTO', 'MUNICIPIO', 'NUMERO_DEL_PREDIO'], inplace=True)
        ## Vamos a construir estas nuevas columnas:
        dataframe['DIVIPOLA'] = dataframe['NPN'].apply(lambda x: str(x)[0:5])
        dataframe['ZONA'] = dataframe['NPN'].apply(lambda x: str(x)[5:7])
        dataframe['SECTOR_COMUNA_BARRIO'] = dataframe['NPN'].apply(lambda x: str(x)[7:13])
        dataframe['MANZANA_VEREDA'] = dataframe['NPN'].apply(lambda x: str(x)[13:17])
        dataframe['TERRENO'] = dataframe['NPN'].apply(lambda x: str(x)[17:21])
        dataframe['CONDICION_PREDIO'] = dataframe['NPN'].apply(lambda x: str(x)[21])
        dataframe['NUMERO_CONSTRUCCION'] = dataframe['NPN'].apply(lambda x: str(x)[22:])
        dataframe['CONDICION_PREDIO'] = dataframe['CONDICION_PREDIO'].map(self.MAPPER_CONDITION)
        dataframe['DESTINO_ECONOMICO'] = dataframe['DESTINO_ECONOMICO'].map(self.MAPPER_DESTI_ECONO)
        dataframe['ZONA'] = dataframe['ZONA'].apply(lambda x: zone_classification(x))
        ### Columns to move
        columns_move = ['DIVIPOLA', 'ZONA', 'SECTOR_COMUNA_BARRIO', 'MANZANA_VEREDA', 'TERRENO', \
                        'CONDICION_PREDIO', 'NUMERO_CONSTRUCCION']
        columns = [dataframe.pop(col) for col in columns_move]
        for idx, col in enumerate(columns):
            dataframe.insert(idx+1, col.name, col)
        dataframe = dataframe.query('NUMERO_DE_ORDEN == TOTAL_REGISTROS')
        return dataframe.reset_index(drop=True)
    ###########################  MÉTODOS DE LA CLASS #########################################
    def get_file_excel(self, folder_to_write, name_file):
        """
        Esta función escribe el pd.DataFrame "arreglado" como un archivo excel.
        """
        name_extension = name_file + '.xlsx'
        df = self._clean_pandas_dataframe()
        path_to_write = os.path.abspath(os.path.join(folder_to_write, name_extension))
        df.to_excel(path_to_write, index=False, sheet_name='NuquiR1')
        return None
    def get_statistical_table(self, columna):

        """
        Esta función proporciona estadísticos básicos para las variables categóricas.
        """
        df = self._clean_pandas_dataframe()    
        grouper = df.groupby(by=[columna]).agg(CANTIDAD_PREDIOS=('NPN', 'count'), AREA_TERRENO=('AREA_TERRENO', 'sum'),
                                            AREA_CONSTRUIDA=('AREA_CONSTRUIDA', 'sum'), AVALUO=('AVALUO','sum'))
        grouper.loc['TOTALES'] = grouper.sum()
        grouper = grouper.sort_values(by=['CANTIDAD_PREDIOS'], ascending=True)
        grouper = grouper.style.format('{:,.0f}')    
        def highlight_row(row):
            styles = [''] * len(row)
            if row.name == 'TOTALES':
                styles = ['background-color: green'] * len(row)
            return styles
        grouper = grouper.apply(highlight_row, axis=1)    
        display(grouper)
        return None
    def get_statistical_plot(self):
        """
        """
        df_plot = self._clean_pandas_dataframe()
        df_plot = pd.pivot_table(df_plot, values=['NPN', 'AREA_TERRENO', 'AREA_CONSTRUIDA', 'AVALUO'], 
                         index=['CONDICION_PREDIO', 'DESTINO_ECONOMICO'], 
                         aggfunc={'AREA_CONSTRUIDA':np.sum, 'NPN': np.size, 'AREA_TERRENO':np.sum, 'AVALUO':np.sum},
                         fill_value=0).reset_index()
        fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'bar'}]])
        conditions = df_plot['CONDICION_PREDIO'].unique()
        labels = {'AREA_CONSTRUIDA': 'Total Área Construida', 'AREA_TERRENO': 'Total Área Terreno', 'AVALUO': 'Total Avalúo',
                'NPN': 'Cantidad'}
        ###
        for columna in ['AREA_CONSTRUIDA', 'AREA_TERRENO', 'AVALUO', 'NPN']:
            for condition in conditions:
                df = df_plot[df_plot['CONDICION_PREDIO'] == condition]
                etiqueta = f"{labels[columna]}"
                fig.add_trace(go.Bar(x=df['DESTINO_ECONOMICO'], y=df[columna], name=etiqueta), 1, 1)
        fig.update_layout(
            title_text="Descriptivos base catastral R1", title_font=dict(size=35, family='Ubuntu', ),#style='italic'),
            template='plotly_dark',
            updatemenus=[{
            'buttons': [
                {'method': 'update', 'label': condition, 'args': [{'visible': [True if cond == condition else False for cond in conditions]}]}
                for condition in conditions
            ],
            'direction': 'down',
            'showactive': True,
            'active': 0,
            'font': dict(color='orange'), 
            'x':1.21,
            'y':0.1
        }])

        fig.show()
        return None

## ***Uso de `ManipulationPyR1`***

In [2]:
## Creamos una instancia de la class ManipulationPyR1
nuquiR1 = ManipulationPyR1('data/nuqui_curso.csv')

### ***Escribimos el Excel "mejorado".***

In [3]:
nuquiR1.get_file_excel('data', 'test1')
print([file for file in os.listdir('data') if '.xlsx' in file])

['nuqui_r1_plot.xlsx', 'clean_nuqui_r1.xlsx', 'DESTINACION_ECONOMICA_CODIGOS.xlsx', 'test1.xlsx']


### ***Tablas estadísticas.***

In [4]:
nuquiR1.get_statistical_table('ZONA')
nuquiR1.get_statistical_table('CONDICION_PREDIO')
nuquiR1.get_statistical_table('DESTINO_ECONOMICO')

,CANTIDAD_PREDIOS,AREA_TERRENO,AREA_CONSTRUIDA,AVALUO
ZONA,,,,
rural,574,"779,626,892","12,012","62,767,119,500"
corregimiento,808,"360,754","40,569","12,940,798,000"
urbano,"1,307","730,955","83,172","57,777,422,000"
TOTALES,"2,689","780,718,601","135,753","133,485,339,500"


,CANTIDAD_PREDIOS,AREA_TERRENO,AREA_CONSTRUIDA,AVALUO
CONDICION_PREDIO,,,,
MejoraPredioAjeno,987,0,"74,707","3,191,003,000"
NoRegPH,"1,702","780,718,601","61,046","130,294,336,500"
TOTALES,"2,689","780,718,601","135,753","133,485,339,500"


,CANTIDAD_PREDIOS,AREA_TERRENO,AREA_CONSTRUIDA,AVALUO
DESTINO_ECONOMICO,,,,
Cultural,1,371,108,"65,999,000"
Recreacional,1,"1,145",0,"52,218,000"
Lote No Urbanizable,2,"286,120",0,"2,555,217,000"
Forestal,3,"169,624,742",0,"11,301,371,000"
Pecuario,3,109,28,"12,799,000"
Servicios Especiales,4,"8,265",20,"8,930,000"
Uso Público,4,"59,053",880,"6,842,368,000"
Salubridad,5,189,359,"48,666,000"
Religioso,6,678,940,"97,356,000"


### ***Un plot de descriptivos.***

In [5]:
nuquiR1.get_statistical_plot()

## ***Plots Base R1.***

***¿Qué se debe conocer en las tablas R1?***

Vamos a pensar en que información deseamos conocer: 

1. Cantidad de predios con área mayor de una hectarea. NO SE REALIZÓ.
2. Cantidad de predios urbanos y rurales. YA
3. Cantidad de predios agregado por destino económico y condición de predio. YA
4. Total avalúo, agredado por zonas del municipio. YA
5. Estadísticas básicas: Tendencia de los valores de los precios teniendo en cuentas los destinos económicos. NO HAY HISTÓRICO
6. Interesaría saber los predios de mayor avalúo dentro de determinada zona de un municipio, junto con su tendencia. NO HAY HISTÓRICO
7. 
9. 
10. 

---

## ***Conclusiones.***

---

Colocamos en un solo artefacto (`class en python`) todo el procesamiento desarrollado en las primeras lecturas, agregamos informaciones descriptivas de caracter tabular y gráfico. La idea central es: `make it easy`. Piense si puede automatizar su trabajo y ponga en marcha esa idea...

---
